In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import numpy as np

Creating DataBase

In [ ]:

#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='Snowdav3', host='127.0.0.1', port= '5432'
)

conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Preparing query to create a database
sql = '''CREATE database taxis''';

#Creating a database
cursor.execute(sql)
print("Database created successfully........")

#Closing the connection
conn.close()

Conecting to DataBase

In [ ]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [ ]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

In [ ]:
conn = connect(param_dic)

Creating Table in the DataBase

In [ ]:
def creating_table():
    cursor = conn.cursor()
    
    sql ='''CREATE TABLE yellowtaxis(
       VendorID float,
       tpep_pickup_datetime timestamp,
       tpep_dropoff_datetime timestamp,
       passenger_count float,
       trip_distance float,
       RatecodeID float,
       store_and_fwd_flag varchar,
       PULocationID float,
       DOLocationID float,
       payment_type float,
       fare_amount float,
       extra float,
       mta_tax float,
       tip_amount float,
       tolls_amount float,
       improvement_surcharge float,
       total_amount float,
       congestion_surcharge float
    );'''
    
    cursor.execute(sql)
    print("Table created")
    conn.commit()
    cursor.close()
    print("Cursor object closed")
    conn.rollback()
    conn.close()
    print("conn closed")

In [ ]:
creating_table()

In [ ]:
months_csv = pd.read_csv('2020_January_February_March_Yellow.csv')
months_csv

In [ ]:
months_csv.dtypes

In [ ]:
type_object = months_csv.select_dtypes(include='object').columns.to_list()
type_object.pop()#.astype('str').dtypes


for elem in type_object:
    months_csv[f"{elem}"] = pd.to_datetime(months_csv[f"{elem}"], yearfirst=True, format="%Y/%m/%d")
months_csv.dtypes

In [ ]:
months_csv

In [ ]:
conn = connect(param_dic)

In [ ]:
def execute_batch(conn, df, table, page_size=100):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    
    print("Converting DataFrame rows into tuples")
    tuples = [tuple(x) for x in df.to_numpy()]
    print("Creating columns")
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    print("Creating cursor")
    cursor = conn.cursor()
    print("Cursor created")
    try:
        print(f"Inserting values into the {table} table")
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Ha entrado en el except")
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print(f"Batch executed succesfully, all dataframe values inserted into the {table} table at {cursor.connection}")
    cursor.close()
    

In [ ]:
execute_batch(conn, months_csv, 'yellowtaxis')

In [ ]:
conn.rollback()
#conn.close()

In [ ]:
import pandas as pd
fg = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
fg

In [ ]:
tuples = [tuple(x) for x in fg.to_numpy()]

In [ ]:
tuples